<a href="https://colab.research.google.com/github/mfikrualbayyan/pembelajaran_mesin/blob/main/Minggu7_Distributed_XGBoost_with_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ray


     |████████████████████████████████| 54.7 MB 30 kB/s 
     |████████████████████████████████| 72 kB 600 kB/s 


In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 36.0 MB/s 


In [3]:
pip install sklearn


In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 12:07:04,424	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-12 12:07:06,660	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=286) [12:07:06] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=286) [12:07:06] Tree method is automatically selected to be 'approx' for distributed training.
(pid=310) [12:07:06] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 12:07:13,512	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-12 12:07:15,677	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-12 12:07:30,149	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-12 12:07:30,170	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-12 12:07:30,172	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_1c1ed_00000,PENDING,,0.00264273,5,0.649034
train_model_1c1ed_00001,PENDING,,0.00404286,7,0.535964
train_model_1c1ed_00002,PENDING,,0.000139259,6,0.897684
train_model_1c1ed_00003,PENDING,,0.00271159,7,0.891078


(pid=390) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=390)   warnings.warn(LEGACY_WARNING)
(pid=390) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=390)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=390) 2021-11-12 12:07:32,517	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=390) 2021-11-12 12:07:35,232	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Trial name,status,loc,eta,max_depth,subsample
train_model_1c1ed_00000,RUNNING,172.28.0.2:390,0.00264273,5,0.649034
train_model_1c1ed_00001,PENDING,,0.00404286,7,0.535964
train_model_1c1ed_00002,PENDING,,0.000139259,6,0.897684
train_model_1c1ed_00003,PENDING,,0.00271159,7,0.891078


Result for train_model_1c1ed_00000:
  date: 2021-11-12_12-07-36
  done: false
  experiment_id: abd882ebd42f4390ae3305f2196771c3
  hostname: 8f78660d33d7
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 390
  time_since_restore: 3.8110313415527344
  time_this_iter_s: 3.8110313415527344
  time_total_s: 3.8110313415527344
  timestamp: 1636718856
  timesteps_since_restore: 0
  train-error: 0.02812
  train-logloss: 0.690853
  training_iteration: 1
  trial_id: 1c1ed_00000
  
Result for train_model_1c1ed_00000:
  date: 2021-11-12_12-07-36
  done: true
  experiment_id: abd882ebd42f4390ae3305f2196771c3
  experiment_tag: 0_eta=0.0026427,max_depth=5,subsample=0.64903
  hostname: 8f78660d33d7
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 390
  time_since_restore: 3.902693271636963
  time_this_iter_s: 0.0065610408782958984
  time_total_s: 3.902693271636963
  timestamp: 1636718856
  timesteps_since_restore: 0
  train-error: 0.019332
  train-logloss: 0.67107
  training_iterati

...

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_1c1ed_00003,RUNNING,172.28.0.2:704,0.00271159,7,0.891078,,,,
train_model_1c1ed_00000,TERMINATED,172.28.0.2:390,0.00264273,5,0.649034,10,3.90269,0.67107,0.019332
train_model_1c1ed_00001,TERMINATED,172.28.0.2:494,0.00404286,7,0.535964,10,3.78052,0.660387,0.022847
train_model_1c1ed_00002,TERMINATED,172.28.0.2:598,0.000139259,6,0.897684,10,3.89225,0.691921,0.010545


Result for train_model_1c1ed_00003:
  date: 2021-11-12_12-07-53
  done: false
  experiment_id: 2405bad56d1548efb35cdd8a613fafb2
  hostname: 8f78660d33d7
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 704
  time_since_restore: 3.7112832069396973
  time_this_iter_s: 3.7112832069396973
  time_total_s: 3.7112832069396973
  timestamp: 1636718873
  timesteps_since_restore: 0
  train-error: 0.031634
  train-logloss: 0.690786
  training_iteration: 1
  trial_id: 1c1ed_00003
  
Result for train_model_1c1ed_00003:
  date: 2021-11-12_12-07-53
  done: true
  experiment_id: 2405bad56d1548efb35cdd8a613fafb2
  experiment_tag: 3_eta=0.0027116,max_depth=7,subsample=0.89108
  hostname: 8f78660d33d7
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 704
  time_since_restore: 3.775371789932251
  time_this_iter_s: 0.005177021026611328
  time_total_s: 3.775371789932251
  timestamp: 1636718873
  timesteps_since_restore: 0
  train-error: 0.008787
  train-logloss: 0.669765
  training_iterat

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_1c1ed_00000,TERMINATED,172.28.0.2:390,0.00264273,5,0.649034,10,3.90269,0.67107,0.019332
train_model_1c1ed_00001,TERMINATED,172.28.0.2:494,0.00404286,7,0.535964,10,3.78052,0.660387,0.022847
train_model_1c1ed_00002,TERMINATED,172.28.0.2:598,0.000139259,6,0.897684,10,3.89225,0.691921,0.010545
train_model_1c1ed_00003,TERMINATED,172.28.0.2:704,0.00271159,7,0.891078,10,3.77537,0.669765,0.008787


(pid=704) 2021-11-12 12:07:54,016	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.83 seconds (1.15 pure XGBoost training time).
2021-11-12 12:07:54,182	INFO tune.py:630 -- Total run time: 24.04 seconds (23.84 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.002711587778961428, 'subsample': 0.8910776376722229, 'max_depth': 7}
